In [8]:
import numpy as np
from numba import types, njit,int64,jit,cuda,prange,vectorize
from numba.typed import List as NumbaList
from BPmodule.BPGen import GetInstance
from BPnumba.GeneticOperators import MutateC2
from BPnumba.DFFA import createDFFA
from BPnumba.EDFFA import createEDFFA,FFSearch
import matplotlib.pyplot as plt
from BPnumba.BoxN import create_ItemBin,ItemBin,InstaceBoxes
from BPnumba.Poblation import CreateRotHPob,EvalPob

In [9]:
def zerovec(y, x): return np.zeros(shape=[x, y], dtype=np.float64)
createText = lambda x,y : np.savetxt(
    fname = "Results/"+x + ".csv",
    X= y,
    delimiter=" ",
    fmt='%g')

In [54]:
@njit#(parallel=True)
def BenchFFA(i,boxes,bin,lst,lst2,lst3,maxItr,pop,rot):
    #fi = NumbaList(np.zeros(maxItr,dtype=np.float64))
    UBOXES = InstaceBoxes(boxes)
    pob = CreateRotHPob(pop =pop, ITEMS = UBOXES, rotType=rot)
    
    EvalPob(pob= pob,boxData=UBOXES,container=bin,rotation=rot)
    
    ind = FFSearch(mutType=0,upIndex=4,Maxitr=maxItr,fireflyPob=pob.copy(),datos=UBOXES.copy(),contenedor=bin,rot=rot)   
    ind2 = FFSearch(mutType=1,upIndex=4,Maxitr=maxItr,fireflyPob=pob.copy(),datos=UBOXES.copy(),contenedor=bin,rot=rot)   
    ind3 = FFSearch(mutType=2,upIndex=4,Maxitr=maxItr,fireflyPob=pob.copy(),datos=UBOXES.copy(),contenedor=bin,rot=rot)    

    lst[i]=ind.fi
    lst2[i] = ind2.fi
    lst3[i] = ind3.fi

In [64]:
@njit(parallel=True)
def BFFA(rango,rango2,contenedor,data:list[list[list[int]]],agL,agL2,agL3,maxItr,numpop,rotation):
    for p in prange(rango,rango2):
        BenchFFA(
                i=p,
                boxes=data[p], 
                bin=contenedor,
                maxItr= maxItr,
                lst= agL,
                lst2=agL2,
                lst3=agL3,
                pop=numpop,
                rot=rotation)


In [65]:
def Bench(nm,r):
    BR1 = GetInstance(nm)
    agL = zerovec(1, 100)
    agL2 = zerovec(1, 100)
    agL3 = zerovec(1, 100)
    for th in np.arange(0,r,4):
        BFFA(rango=th,rango2=th+4,contenedor=BR1[0],data=BR1[1],agL=agL,agL2=agL2,agL3=agL3,maxItr=1000,numpop=20,rotation=6)
        createText(nm+"/FFA/"+"FFA"+str(0),agL)
        createText(nm+"/FFA/"+"FFA"+str(1),agL2)
        createText(nm+"/FFA/"+"FFA"+str(2),agL3)

In [66]:
r=100
agL = zerovec(1, 100)
agL2 = zerovec(1, 100)
agL3 = zerovec(1, 100)
BR1 = GetInstance("P2A2")
cont = BR1[0]
dataBoxes=BR1[1]
boxes = dataBoxes[0]

In [60]:
 BenchFFA(
            i=0,
            boxes=boxes, 
            bin=cont,
            maxItr= 1000,
            lst= agL,
            lst2=agL2,
            lst3=agL3,
            pop=20,
            rot=6)

In [68]:
Bench("P2A2",100)

In [ ]:
Bench("P3A2",100)

In [ ]:
pop=20
maxItr=1000
rot=6
m=4
mut=0

In [ ]:
BR1 = GetInstance("P2A2")
cont = BR1[0]
dataBoxes=BR1[1]
boxes = dataBoxes[0]

In [ ]:
UBOXES = InstaceBoxes(boxes)
pob = CreateRotHPob(pop =pop, ITEMS = UBOXES, rotType=rot)
EvalPob(pob,UBOXES,cont,rot)

In [ ]:
ind = FFSearch(mutType=mut,upIndex=m,Maxitr=maxItr,fireflyPob=pob.copy(),datos=UBOXES.copy(),contenedor=cont,rot=6)

In [ ]:
ind2 = FFSearch(mutType=2,upIndex=m,Maxitr=maxItr,fireflyPob=pob.copy(),datos=UBOXES.copy(),contenedor=cont,rot=6)


In [ ]:
ind1 = FFSearch(mutType=1,upIndex=m,Maxitr=maxItr,fireflyPob=pob.copy(),datos=UBOXES.copy(),contenedor=cont,rot=6)


In [ ]:
ind.fi,ind1.fi,ind2.fi